In [ ]:
import os
import pandas as pd
from glob import glob
root = r'F:\20240120-HanDanDan\patches'

patches = pd.DataFrame(glob(os.path.join(root, '*', '*.jpg')), columns=['fpath'])
patches['ID'] = patches['fpath'].map(lambda x: os.path.basename(os.path.dirname(x)))
patches['filename'] = patches['fpath'].map(lambda x: os.path.basename(x))
patches

In [ ]:
import os
import shutil
import pandas as pd
from onekey_algo.custom.components.comp2 import split_dataset4sol
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.utils import print_join_info

data_root = get_param_in_cwd('data_root')
data = pd.read_csv(os.path.join(data_root, 'label.csv'), dtype={'ID': str})
train_data = data[data['group'] == 'train']
test_data = data[data['group'] != 'train']

rt = split_dataset4sol(train_data, train_data['label'], cv=False, n_trails=10, test_size=0.33, save_dir=data_root, shuffle=True)
for idx, (train, val) in enumerate(rt):    
    val['group'] = 'val'
    pd.concat([train, val, test_data], axis=0).to_csv(os.path.join(data_root, 'split_info', f'label-RND-{idx}.csv'), index=False)
    train = pd.merge(train, patches[['ID', 'fpath']], on='ID', how='inner')
    print('训练集:', len(set(train['ID'])))
    train[['fpath', 'label']].to_csv(os.path.join(data_root, 'split_info', f'train-RND-{idx}.txt'), index=False, header=False, sep='\t')
    val['group'] = 'val'
    val = pd.concat([val, test_data], axis=0)
    val = pd.merge(val, patches[['ID', 'fpath']], on='ID', how='inner')
    val[['fpath', 'label']].to_csv(os.path.join(data_root, 'split_info', f'val-RND-{idx}.txt'), index=False, header=False, sep='\t')
    
    print('测试集:', len(set(val['ID'])))